# Private Set Intersection and Moose

This notebook takes a common use case: sharing data between several parties in order to compute something together.

In this case, you will use [Facebook's PrivateID library](https://github.com/facebookresearch/Private-ID#private-id-1) to compute a join between the two datasets and then [Moose from the tf-encrypted team](https://github.com/tf-encrypted/moose) to compute the values using encrypted data.

For this particular example, the players are two companies with some overlap in customers. They would like to find customers that are of interest for new product offerings based on the spending categories of their combined customer bases. They can then use this result to determine what product offerings would be most appealing to which customers and reach out for more customer research.

This notebook was heavily based on [Yann Dupis' notebooks](https://github.com/yanndupis) with additional support and Moose fixes from [Jason Mancuso](https://github.com/jvmncs). I encourage you to follow their work on GitHub to keep up with the latest!


## Generate fake datasets

In this example, you'll have Alice and Bob play the two data scientists at each company. They need to first have realistic data, so here you'll generate some example data. Feel free to play around with the generator to test out new ideas!

In [1]:
import pathlib
import numpy as np
import pandas as pd

np.random.seed(1234)

_DATA_DIR = pathlib.Path("./data")

def generate_mock_dataset(sample_size, n_features, sample_frac, 
                          scaler=100, positive=True, precision=4):
    x =  np.random.randn(sample_size, n_features).round(precision) * scaler
    user_id = np.array(range(sample_size))
    df = pd.DataFrame(x, columns=[f"x_{i}" for i in range(n_features)])
    df.insert(loc=0, column='user_id', value=user_id)
    df =  df.sample(frac=sample_frac)
    if positive:
        df = df.applymap(lambda x: x * -1 if x < 0 else x)
    return df

alice_df = generate_mock_dataset(10, 20, 0.6)
bob_df = generate_mock_dataset(10, 1, 0.7)

In [2]:
alice_df

,user_id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19
1,1,20.26,65.60,19.34,55.34,131.82,46.93,67.56,181.70,18.31,...,39.78,33.74,104.76,104.59,86.37,12.21,12.47,32.28,84.17,239.10
0,0,47.14,119.10,143.27,31.27,72.06,88.72,85.96,63.65,1.57,...,115.00,99.19,95.33,202.13,33.41,0.21,40.55,28.91,132.12,154.69
8,8,53.68,74.38,32.02,91.62,85.97,22.60,62.88,18.65,95.25,...,7.26,55.06,93.82,123.91,13.97,22.30,212.37,12.23,140.94,142.30
2,2,7.62,56.64,3.61,207.50,24.78,89.72,13.68,1.83,75.54,...,84.10,144.58,140.20,10.09,54.82,14.46,35.40,3.55,56.57,154.57
5,5,29.12,56.65,50.36,28.53,48.43,136.35,78.11,46.80,122.46,...,87.55,171.07,45.08,74.92,20.39,18.22,68.07,181.85,4.71,39.48
4,4,46.44,356.35,132.11,15.26,16.45,43.01,76.74,98.49,27.08,...,7.98,40.00,102.79,58.47,81.66,8.19,34.48,52.83,106.90,51.19


### Run Private-ID

Now it's time to run [Private-ID](https://github.com/facebookresearch/Private-ID#private-id-1) on Alice and Bob's User IDs. You will want to install Private-ID in a parent folder near this repository and run these commands from the location where you installed Private ID. Make sure to update the input and output file paths should your location be different!

```
env RUST_LOG=info cargo run --bin private-id-server -- \
    --host 0.0.0.0:10009 \
    --input ../practical-data-privacy/data/alice_id.csv \
    --output ../practical-data-privacy/data/alice_keys.csv \
    --no-tls
    

env RUST_LOG=info cargo run --bin private-id-client -- \
    --company localhost:10009 \
    --input ../practical-data-privacy/data/bob_id.csv \
    --output ../practical-data-privacy/data/bob_keys.csv  \
    --no-tls
```

In [3]:
alice_id = alice_df["user_id"]
alice_id.to_csv(_DATA_DIR / "alice_id.csv", index=False, header=False)

bob_id = bob_df["user_id"]
bob_id.to_csv(_DATA_DIR / "bob_id.csv", index=False, header=False)

Here is below the outputs from Private-ID. As you can see they both contain the same list of keys ordered the same way but for some of them they have a User Id mapped to them. 

In [4]:
alice_keys = pd.read_csv(_DATA_DIR / "alice_keys.csv", names=["key", "user_id"])

alice_keys

,key,user_id
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,4.0
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,NaN
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,NaN
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0


In [5]:
bob_keys = pd.read_csv(_DATA_DIR / "bob_keys.csv", names=["key", "user_id"])

bob_keys

,key,user_id
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,NaN
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,7.0
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,9.0
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0


We then merge the Private-ID output to Alice and Bob's datasets so in the datasets we have the all the keys from Private-ID with the corresponding User ID and features.

In [6]:
alice_df = pd.merge(alice_keys, alice_df, on='user_id', how='left')
bob_df = pd.merge(bob_keys, bob_df, on='user_id', how='left')

In [7]:
alice_df

,key,user_id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0,7.62,56.64,3.61,207.50,24.78,89.72,13.68,1.83,...,84.10,144.58,140.20,10.09,54.82,14.46,35.40,3.55,56.57,154.57
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0,53.68,74.38,32.02,91.62,85.97,22.60,62.88,18.65,...,7.26,55.06,93.82,123.91,13.97,22.30,212.37,12.23,140.94,142.30
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0,20.26,65.60,19.34,55.34,131.82,46.93,67.56,181.70,...,39.78,33.74,104.76,104.59,86.37,12.21,12.47,32.28,84.17,239.10
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,4.0,46.44,356.35,132.11,15.26,16.45,43.01,76.74,98.49,...,7.98,40.00,102.79,58.47,81.66,8.19,34.48,52.83,106.90,51.19
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0,29.12,56.65,50.36,28.53,48.43,136.35,78.11,46.80,...,87.55,171.07,45.08,74.92,20.39,18.22,68.07,181.85,4.71,39.48
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0,47.14,119.10,143.27,31.27,72.06,88.72,85.96,63.65,...,115.00,99.19,95.33,202.13,33.41,0.21,40.55,28.91,132.12,154.69


In [8]:
bob_df

,key,user_id,x_0
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0,0.31
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0,89.36
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0,142.85
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,NaN,NaN
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,7.0,52.65
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0,30.27
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,9.0,40.11
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0,87.34


In each dataset, we create a flag `user_id_available` by checking if the user id is missing or not. For the column feature x_0, we replace the missing values with 0. The value doesn't matter since these records will be filtered out when taking the intersection.

In [9]:
bob_df['user_id_available'] = np.where(bob_df['user_id'].isnull(), 0, 1)
bob_df = bob_df.fillna(0)
bob_df

,key,user_id,x_0,user_id_available
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0,0.31,1
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0,89.36,1
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0,142.85,1
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,0.0,0.00,0
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,7.0,52.65,1
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0,30.27,1
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,9.0,40.11,1
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0,87.34,1


In [10]:
alice_df['user_id_available'] = np.where(alice_df['user_id'].isnull(), 0, 1)
alice_df = alice_df.fillna(0)
alice_df

,key,user_id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,user_id_available
0,144418D81867C352A318B3C8BE1FC84BE959FC51FADF4E...,2.0,7.62,56.64,3.61,207.50,24.78,89.72,13.68,1.83,...,144.58,140.20,10.09,54.82,14.46,35.40,3.55,56.57,154.57,1
1,5CDC4E5348AA5E4DECC5F0AAB4DFB73D09C2BD4E4ED28B...,8.0,53.68,74.38,32.02,91.62,85.97,22.60,62.88,18.65,...,55.06,93.82,123.91,13.97,22.30,212.37,12.23,140.94,142.30,1
2,72A1523C8A9575605E886810AAB545E9957EB22DE5CCBC...,1.0,20.26,65.60,19.34,55.34,131.82,46.93,67.56,181.70,...,33.74,104.76,104.59,86.37,12.21,12.47,32.28,84.17,239.10,1
3,72F5125B88854DDAF88DA9D225A348AC8B4E3DCE6F097F...,4.0,46.44,356.35,132.11,15.26,16.45,43.01,76.74,98.49,...,40.00,102.79,58.47,81.66,8.19,34.48,52.83,106.90,51.19,1
4,8A623CCEE51534754EAE9D65F66ADE66BF51E241783FDA...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
5,92E45747889DB6D24755537D369BF68D384DC9549F1D77...,5.0,29.12,56.65,50.36,28.53,48.43,136.35,78.11,46.80,...,171.07,45.08,74.92,20.39,18.22,68.07,181.85,4.71,39.48,1
6,F225409B2CBC3F22CD8EBCACDE75447816E81BD9DB363E...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
7,F438B71D793E7CD9C2A6A0DF5B0AE501F84833525237A2...,0.0,47.14,119.10,143.27,31.27,72.06,88.72,85.96,63.65,...,99.19,95.33,202.13,33.41,0.21,40.55,28.91,132.12,154.69,1


### Moose Computation

To the Moose computation we will feed Alice's `user_id_available` flag and feature and Bob's `user_id_available` flag and feature.

We do the intersection using a logical And operation between `user_id_available` flags from Alice and Bob, filter the datasets where the logical And returns 1, then compute a metric privately.

In [12]:
user_id_available_a = alice_df.user_id_available.values
user_id_available_b = bob_df.user_id_available.values

alice_features_for_computation = alice_df.drop(['key', 'user_id', 'user_id_available'], axis=1)
bob_features_for_computation = bob_df.drop(['key', 'user_id', 'user_id_available'], axis=1)

x_a = alice_features_for_computation.values
x_b = bob_features_for_computation.values

In [13]:
np.save(_DATA_DIR / "x_a", x_a)
np.save(_DATA_DIR / "x_b", x_b)
np.save(_DATA_DIR / "user_id_available_a", user_id_available_a)
np.save(_DATA_DIR / "user_id_available_b", user_id_available_b)

In [14]:
import pymoose as pm

FIXED = pm.fixed(24, 40)

alice = pm.host_placement(name="alice")
bob = pm.host_placement(name="bob")
carole = pm.host_placement(name="carole")
rep = pm.replicated_placement(name="rep", players=[alice, bob, carole])
mirrored = pm.mirrored_placement(name="mirrored", players=[alice, bob, carole])

@pm.computation
def psi_and_agg():    
    with alice:
        x_a = pm.load("x_a", dtype=pm.float64)
        user_id_available_a = pm.load("user_id_available_a", dtype=pm.bool_)

    with bob:
        x_b = pm.load("x_b", dtype=pm.float64)
        user_id_available_b = pm.load("user_id_available_b", dtype=pm.bool_)

        # # Compute logical And between user_id_available from Alice and Bob.
        # # If it returns 1, it means the User ID was in Alice and Bob's datasets
        exist_in_alice_and_bob_bool = pm.logical_and(
            user_id_available_a, user_id_available_b
        )

        # # Filter Bob's feature to keep only records where exist_in_alice_and_bob_bool returned 1
        x_b_sub = pm.select(x_b, axis=0, index=exist_in_alice_and_bob_bool)
        x_b_sub = pm.cast(x_b_sub, dtype=FIXED)

    with alice:
        # Filter Alice's feature to keep only records where exist_in_alice_and_bob_bool returned 1
        x_a_sub = pm.select(x_a,  axis=0, index=exist_in_alice_and_bob_bool)
        x_a_sub = pm.cast(x_a_sub, dtype=FIXED)

    with mirrored:
        ten_percent = pm.constant(0.1, dtype=FIXED)
        
    with rep:
        # Aggregation: average ratio between sum of x_a_sub & x_b_sub
        spend_per_category = x_a_sub + x_b_sub        
        spend_per_user = pm.sum(spend_per_category, axis=1)
        category_percent = spend_per_category / pm.expand_dims(spend_per_user, axis=1)
        res = pm.greater(category_percent, ten_percent)

    with alice:
        res = pm.cast(res, dtype=pm.float64)
        res = pm.save("agg_result", res)

    return res

In [15]:
executors_storage = {
    "alice": {"x_a": x_a, "user_id_available_a": user_id_available_a.astype(np.bool_)},
    "bob": {"x_b": x_b, "user_id_available_b": user_id_available_b.astype(np.bool_)}
}

runtime = pm.LocalMooseRuntime(
    identities=["alice", "bob", "carole"],
    storage_mapping=executors_storage,
)

runtime.set_default()

_ = psi_and_agg()

agg_result = runtime.read_value_from_storage("alice", "agg_result")
print("Aggregation result with Moose", agg_result)

Aggregation result with Moose [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [30]:
# In plaintext


inner_bool = np.logical_and(user_id_available_a, user_id_available_b)
x_a_sub = x_a[np.where(inner_bool==1)]
x_b_sub = x_b[np.where(inner_bool==1)]

spend_per_category = x_a_sub + x_b_sub
spend_per_user = spend_per_category.sum(axis=1)
category_percent = spend_per_category / np.expand_dims(spend_per_user, axis=1)
plaintext_result = np.greater(category_percent, 0.1).astype(int) 
plaintext_result

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [31]:
plaintext_result == agg_result

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]])

## Challenges

- Adapt the computation so that it also returns the result to Bob
- Try out a new computation -- what if they wanted to find the categories where their shared customers spend the least amount of money?
- Add more rows and see how size affects the computation.
- Try out another Private-ID or Moose computation by building your own example. Feel free to contribute examples to the reader-examples folder!